<a href="https://colab.research.google.com/github/joris-vaneyghen/mss-jazz-playalong/blob/main/explore_audio_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio Segmentation & Musical Instrument Tagger

### Objective:
The goal of this project is to segment an audio file into distinct sections and tag each section with the instruments being played.

### Requirements:
- Each segment should be at least **2,5 seconds** in length.
- Consecutive segments should feature **different sets of instruments**.

In [ ]:
!pip install ruptures

In [ ]:
# download our audio example
!git clone https://github.com/joris-vaneyghen/mss-jazz-playalong.git

In [ ]:
# dowload our audio tagger
!git clone https://github.com/fschmid56/EfficientAT

In [ ]:
# Let's listen to our audio example

from IPython.display import Audio

Audio('mss-jazz-playalong/examples/Jazz Standards Medley.mp3')


In [ ]:
# Using the audio tagger detect the acoustic events in our audio example. This will print the top 10 detected acoustic events (set runtime type to GPU for faster run)
#!cd EfficientAT && python inference.py --cuda --model_name=dymn20_as --audio_path="../mss-jazz-playalong/examples/Jazz Standards Medley.mp3"

### Instrument Detection Limitations:

Our audio tagger successfully detects instruments such as **Singing, Saxophone, Trombone, and Trumpet**, but tends to ignore **drums** and **double bass**. This limitation arises because the tagger was trained on the **Audioset** dataset, which uses **weakly-labeled** data. In this dataset, **drums** and **bass** were often overlooked, leading to reduced detection accuracy for these instruments.


### Segmentation Approach:

To segment the audio, we convert the waveform into a **multi-dimensional time series** of sound class detections using our audio tagger. This results in a time series with **527 dimensions**, each corresponding to one of the sound classes detected by the tagger.

For detecting change points in this time series, we use the **Ruptures** library, which is well-suited for this task due to several reasons:

- **Versatility**: Ruptures can handle a wide range of data types and is adaptable to different segmentation problems, making it ideal for complex multi-dimensional audio data.
- **Efficiency**: It is optimized for large datasets, allowing fast and accurate detection of change points, even when dealing with high-dimensional time series.
- **Customizability**: Ruptures offers a variety of methods (e.g., dynamic programming, window-based detection) that can be tailored to our specific needs, ensuring robust and reliable segmentation.

By using Ruptures, we can effectively identify moments where the instrument set or sound profile changes, leading to precise audio segmentation.


In [ ]:
%cd EfficientAT/

Inspired on EfficientAT/inference.py we load the audio tagger model

In [ ]:
import torch
from models.dymn.model import get_model as get_dymn
from models.preprocess import AugmentMelSTFT
from helpers.utils import NAME_TO_WIDTH


def load_mel_and_dymn20_as(device):
    """
    Load the model and mel spectrogram processor for audio tagging.

    Args:
        device (torch.device): The device to load the model onto (e.g., 'cuda' or 'cpu').

    Returns:
        mel (AugmentMelSTFT): Mel spectrogram processor.
        model (torch.nn.Module): Loaded model.
    """
    sample_rate=32000
    window_size=800
    hop_size=320
    n_mels=128
    strides=[2, 2, 2, 2]
    model_name = 'dymn20_as'

    model = get_dymn(width_mult=NAME_TO_WIDTH(model_name), pretrained_name=model_name, strides=strides)

    # Send model to the specified device
    model.to(device)
    model.eval()

    # Create a mel spectrogram processor (preprocessor)
    mel = AugmentMelSTFT(n_mels=n_mels, sr=sample_rate, win_length=window_size, hopsize=hop_size)
    mel.to(device)
    mel.eval()

    return mel, model


### Customizing Model Output: Retaining the Time Dimension

In our customized model, we aim to keep the **Time** dimension while processing the waveform:

1. **Stereo Channels as Batch**:  
   In the original implementation, a mono waveform is processed. In our case, we use a stereo waveform and stack the left and right channel at the batch dimension.

2. **Time-Frequency Domain Conversion**:  
   After the input waveform is converted into the Time-Frequency domain, the model compresses both the **Time** and **Frequency** dimensions by a factor of 32.

2. **Pooling Operation Before MLP Layers**:  
   Before the final MLP layers, the model performs an **Average Pooling** operation. However, instead of averaging over the Time and Frequency dimensions, we choose to:
   - Retain the **Time** dimension.
   - Perform the averaging across the **Batch** and **Frequency** dimensions.

This approach ensures that the models output can be used as a multi-dimensional time series.


In [ ]:
import librosa
import numpy as np
from torch import autocast
from contextlib import nullcontext

def preds_over_time(mel, model, waveform, device):
  waveform = torch.from_numpy(waveform).to(device) # shape = (C=2,L)
  with torch.no_grad(), autocast(device_type=device.type) if device == 'cuda' else nullcontext():
    spec = mel(waveform) # shape = (C, F=128, T=L/320)
    input = spec.unsqueeze(1) # shape = (N=C, D=1, F, T)
    print(input.shape)
    features = model._feature_forward(input) # shape = (N, D=1920, F'=F/32, T'≃T/32)
    print(features.shape)
    #We permute Time with Batch dimensions so that pooling is done on the batch and frequency dimension
    features = features.permute(3, 1, 2, 0) # shape = (T', F', C', N)
    preds, embed = model._clf_forward(features)
    preds = torch.sigmoid(preds.float()).squeeze().cpu().numpy() # shape = (T', D'=527)
  return preds, embed


In [ ]:
# lets test this on our example
audio_path = '../mss-jazz-playalong/examples/Jazz Standards Medley.mp3'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mel, model = load_mel_and_dymn20_as(device)
(waveform, _) = librosa.core.load(audio_path, sr=32000, mono=False)

preds, embed = preds_over_time(mel, model, waveform, device)

In [ ]:
# The time dimension is reduced by a factor 10240 (hop_size * compress_factor)
import math
assert math.ceil(waveform.shape[1] / (320 * 32)) == preds.shape[0]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(preds[:, 27])
plt.xlabel("Time")
plt.ylabel("Probability of Singing")
plt.title("Probability of Singing over Time")
plt.show()

plt.plot(preds[:, 153])
plt.xlabel("Time")
plt.ylabel("Probability of Piano")
plt.title("Probability of Piano over Time")
plt.show()

plt.plot(preds[:, 197])
plt.xlabel("Time")
plt.ylabel("Probability of Sax")
plt.title("Probability of Sax over Time")
plt.show()


In [ ]:
import ruptures as rpt

# signal = preds[:, [27, 153, 197]]
signal = embed

# Stel de minimale lengte van een segment in op 8 (= 2,56 seconden. namelijk 8 * (32 * 320) / 32000)
min_size = 8

# Gebruik de Pelt-methode voor breekpuntdetectie
model = "rbf"  # Verandering in gemiddelde (kan aangepast worden naar andere methoden zoals "l1", "l2", "rbf")
algo = rpt.Pelt(model=model, min_size=min_size, jump=1).fit(signal)


# Detecteer breekpunten, zonder het aantal vooraf te specificeren
penalty = 2.5  # Penalty bepaalt hoe streng we breekpunten toestaan, je kunt hiermee spelen
bkps = algo.predict(pen=penalty)

# Plot het resultaat
rpt.display(preds[:, 153], bkps, figsize=(10, 6))  # Plot alleen de eerste dimensie
plt.title("Detectie van breekpunten in het multidimensionale signaal")
plt.show()


# Print de gevonden breekpunten
print("# breekpunten:", len(bkps))
print("Gevonden breekpunten:", bkps)


In [ ]:
(waveform, sr) = librosa.core.load(audio_path, mono=True)
f = (10240 * sr) // 32000

In [ ]:
display(Audio(waveform[0: bkps[0] * f], rate=sr))
for i in range(len(bkps)-1):
  display(Audio(waveform[bkps[i] * f: bkps[i+1] * f], rate=sr))